In [20]:
# Imports
import numpy as np
import pandas as pd
import math
from antropy import entropy
import scipy
import scipy.signal
from envelope_derivative_operator.energy_operators import edo
from scipy.signal import periodogram, welch
from math import factorial, log
from sklearn.preprocessing import StandardScaler
from math import *
from  statistics import *
import pandas as pd
import numpy as np
from scipy.stats import variation,iqr

In [21]:
# # Install necessary git repositories and pip installations

# !git clone https://github.com/ktapani/Neonatal_Seizure_Detection.git
# !git clone https://github.com/raphaelvallat/antropy.git
# !git clone https://github.com/otoolej/envelope_derivative_operator.git
# !pip install numba


In [22]:
%%time
# Read your respective window lengths file
df = pd.read_csv('filtered_babydf16sec.csv')

CPU times: total: 1min 25s
Wall time: 1min 58s


In [23]:
df

,0,1,2,3,4,5,6,7,8,9,...,9207,9208,9209,9210,9211,9212,9213,9214,9215,9216
0,0.334070,0.330610,0.326677,0.335233,0.334722,0.331979,0.334578,0.337308,0.339754,0.340219,...,0.681602,0.635466,0.778547,0.632911,0.449898,0.779114,0.698889,0.570441,0.615573,0.0
1,0.548271,0.540298,0.771639,0.564001,0.377428,0.054457,0.278922,0.386835,0.556868,0.647486,...,0.393234,0.521934,0.442202,0.248731,0.573361,0.301476,0.525077,0.537868,0.399181,0.0
2,0.478351,0.521102,0.596553,0.587973,0.590423,0.574175,0.522126,0.401638,0.362345,0.268294,...,0.361714,0.405822,0.570886,0.613411,0.678019,0.275495,0.737498,0.376802,0.536217,0.0
3,0.659554,0.633758,0.631841,0.655408,0.617889,0.619877,0.661458,0.668227,0.662083,0.689966,...,0.829582,0.719980,0.644163,0.804180,0.738719,0.508473,0.707617,0.941304,0.610385,1.0
4,0.235219,0.252174,0.171776,0.196293,0.127655,0.154571,0.001539,0.526797,0.941654,0.727584,...,0.203947,0.551485,0.492322,0.283676,0.286303,0.424888,0.180210,0.062561,0.459432,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0.456309,0.488854,0.352950,0.362895,0.287580,0.345334,0.390683,0.443779,0.566536,0.686846,...,0.400807,0.475916,0.568595,0.540976,0.573775,0.584586,0.395396,0.464161,0.424183,0.0
19370,0.505731,0.527825,0.418474,0.324620,0.356584,0.222151,0.364497,0.478592,0.524489,0.479076,...,0.708344,0.679775,0.680883,0.661693,0.744460,0.598325,0.646481,0.680233,0.539733,0.0
19371,0.589841,0.621403,0.747336,0.730091,0.898227,0.960461,0.919929,0.733236,0.714369,0.652834,...,0.821700,0.777723,0.875823,0.891038,0.940296,0.989177,0.766933,0.795107,0.656214,0.0
19372,0.538782,0.507820,0.531016,0.609154,0.610146,0.657223,0.653565,0.630671,0.622133,0.624766,...,0.512097,0.542772,0.523989,0.554004,0.534998,0.564243,0.544967,0.573367,0.553773,0.0


<ins>Features considered in this notebook:</ins>

Total number of features considered = 11

- Mean
- Standard Deviation
- Kurtosis
- Skewness
- Shannon Entropy
- Activity
- Mobility
- Complexity
- Permutation Entropy
- Sample Entropy
- Approximate Entropy

In [24]:
# Hyperparams
window_length =512


In [25]:
# %%time
# class NeonatalSeizureFeatures:
#     def __init__(self, row):
#         self.row = row
        
#     def skewness(self):
#         row = np.array(self.row)
#         row = row[:-1]
#         row = np.reshape(row, (21, window_length))
#         return (pd.Series(scipy.stats.skew(x, axis = 0, bias = False) for x in row))
    
    
# df_new = df.apply(lambda row: NeonatalSeizureFeatures(row).skewness(), axis = 1)

# df_new

### Helper Methods:

In [26]:
def hMob(x):
    row = np.array(x)
    return (np.sqrt(np.var(np.gradient(x)) / np.var(x)))

### Feature Methods:

In [27]:
# Feature Methods
def feature_median(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(np.median(x, axis = 0) for x in row))
def feature_variation(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(variation(x, axis = 0) for x in row))
def feature_iqr(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(iqr(x, axis = 0) for x in row))

def feature_RMS(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(np.sqrt(np.mean(x, axis = 0)) for x in row))

def feature_mean(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(np.mean(x, axis = 0) for x in row))

def feature_stddev(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(np.std(x, axis = 0) for x in row))

def kurtosis(row):
    row = np.array(row)
    annotation = row[-1]
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(scipy.stats.kurtosis(x, axis = 0, bias = False) for x in row))

def skewness(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(scipy.stats.skew(x, axis = 0, bias = False) for x in row))
    
def spectral_entropy(row, sf = 32, nperseg = window_length, axis = 1):
    row = np.array(row)
    annotation = row[-1]
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    _, psd = welch(row, sf, nperseg=nperseg, axis=axis)
    psd_norm = psd / psd.sum(axis=axis, keepdims=True)
    se = - np.where(psd_norm == 0, 0, psd_norm * np.log(psd_norm) / np.log(2)).sum(axis=axis)
    return pd.Series(se)

def hjorthActivity(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(np.var(x, axis = 0) for x in row))

def hjorthMobility(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(np.sqrt(np.var(np.gradient(x)) / np.var(x)) for x in row))

def hjorthComplexity(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series((hMob(np.gradient(x)) / hMob(x)) for x in row))

def permutation_entropy(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(entropy.perm_entropy(x) for x in row))

def sample_entropy(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(entropy.sample_entropy(x) for x in row))

def approximate_entropy(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (18, window_length))
    return (pd.Series(entropy.app_entropy(x) for x in row))

In [28]:
list_of_feature_methods = [feature_median,feature_variation,feature_iqr,feature_RMS,feature_mean,feature_stddev, kurtosis, skewness, spectral_entropy, hjorthActivity, hjorthMobility, 
                          hjorthComplexity, permutation_entropy, sample_entropy, approximate_entropy]

In [29]:
%%time
df_list = list()
for i, j in zip(list_of_feature_methods, range(len(list_of_feature_methods))):
    print("Epoch %d ..." % (j+1))
    df_temp = df.apply(lambda row: i(row), axis = 1)
    df_list.append(df_temp)

new_df = pd.concat(df_list, axis = 1)
new_df

Epoch 1 ...
Epoch 2 ...
Epoch 3 ...
Epoch 4 ...
Epoch 5 ...
Epoch 6 ...
Epoch 7 ...
Epoch 8 ...
Epoch 9 ...
Epoch 10 ...
Epoch 11 ...
Epoch 12 ...
Epoch 13 ...
Epoch 14 ...
Epoch 15 ...
CPU times: total: 1h 3min 40s
Wall time: 1h 12min 11s


,0,1,2,3,4,5,6,7,8,9,...,8,9,10,11,12,13,14,15,16,17
0,0.335164,0.422834,0.425434,0.386857,0.268384,0.364790,0.372324,0.337670,0.317918,0.323084,...,0.562916,0.547509,0.428001,0.381886,0.538637,0.495738,0.543177,0.453424,0.320640,1.318853
1,0.558940,0.547185,0.536815,0.439565,0.635416,0.497883,0.601128,0.535507,0.491099,0.487306,...,1.182261,1.198504,1.258176,1.284958,1.187919,1.312455,1.242558,1.224121,1.272901,1.324214
2,0.481070,0.521079,0.499746,0.613020,0.501244,0.668536,0.593137,0.620548,0.625243,0.594081,...,1.165262,1.187891,1.316769,1.341679,1.294233,1.312466,1.173382,1.296258,1.354735,1.374708
3,0.648244,0.654992,0.654824,0.651539,0.498437,0.690035,0.509310,0.669250,0.483494,0.501346,...,0.864229,0.837796,0.424594,0.528106,0.817291,0.404327,0.634072,0.440675,0.732804,1.354924
4,0.273979,0.277746,0.284761,0.281482,0.303147,0.285764,0.280198,0.274268,0.269927,0.271021,...,0.714092,0.690006,0.812859,0.717739,0.951507,0.772926,0.886945,0.730272,0.576406,1.311020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0.420159,0.539634,0.553114,0.445364,0.456413,0.484084,0.491127,0.502279,0.405376,0.530856,...,1.338880,1.272673,1.261968,1.310505,1.349567,1.252342,1.353315,1.225385,1.336892,1.304636
19370,0.481627,0.467439,0.593716,0.594382,0.465957,0.529134,0.553749,0.482576,0.462229,0.442401,...,1.390582,1.268374,1.322265,1.341143,1.346202,1.294336,1.359204,1.328350,1.340989,1.352717
19371,0.623665,0.564269,0.519484,0.492092,0.561203,0.471161,0.608575,0.581709,0.579173,0.511012,...,1.314256,1.299334,1.309861,1.358664,1.271001,1.388717,1.361720,1.295100,1.296166,1.350382
19372,0.548308,0.526311,0.558804,0.603206,0.437835,0.499259,0.615961,0.377607,0.531348,0.441672,...,1.214614,1.347398,1.200716,1.251561,1.318744,1.269002,1.226858,1.221599,1.275983,1.329643


In [30]:
# %%time


# df_list = list()
# for i, j in zip(list_of_feature_methods, range(len(list_of_feature_methods))):
    
#     print("Epoch %d ..." % (j+1))
#     df_temp = df.apply(lambda row: i(row), axis = 1)
#     df_temp .reset_index(name='%s%d'%i%j)

#     #print(i)
#     df_list.append(df_temp)
#     #print(df_temp)

# new_df = pd.concat(df_list, axis = 1)
# new_df

In [31]:
feature_df8 = pd.concat([new_df, df[df.columns[-1]]], axis = 1)
feature_df8

,0,1,2,3,4,5,6,7,8,9,...,9,10,11,12,13,14,15,16,17,9216
0,0.335164,0.422834,0.425434,0.386857,0.268384,0.364790,0.372324,0.337670,0.317918,0.323084,...,0.547509,0.428001,0.381886,0.538637,0.495738,0.543177,0.453424,0.320640,1.318853,0.0
1,0.558940,0.547185,0.536815,0.439565,0.635416,0.497883,0.601128,0.535507,0.491099,0.487306,...,1.198504,1.258176,1.284958,1.187919,1.312455,1.242558,1.224121,1.272901,1.324214,0.0
2,0.481070,0.521079,0.499746,0.613020,0.501244,0.668536,0.593137,0.620548,0.625243,0.594081,...,1.187891,1.316769,1.341679,1.294233,1.312466,1.173382,1.296258,1.354735,1.374708,0.0
3,0.648244,0.654992,0.654824,0.651539,0.498437,0.690035,0.509310,0.669250,0.483494,0.501346,...,0.837796,0.424594,0.528106,0.817291,0.404327,0.634072,0.440675,0.732804,1.354924,1.0
4,0.273979,0.277746,0.284761,0.281482,0.303147,0.285764,0.280198,0.274268,0.269927,0.271021,...,0.690006,0.812859,0.717739,0.951507,0.772926,0.886945,0.730272,0.576406,1.311020,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0.420159,0.539634,0.553114,0.445364,0.456413,0.484084,0.491127,0.502279,0.405376,0.530856,...,1.272673,1.261968,1.310505,1.349567,1.252342,1.353315,1.225385,1.336892,1.304636,0.0
19370,0.481627,0.467439,0.593716,0.594382,0.465957,0.529134,0.553749,0.482576,0.462229,0.442401,...,1.268374,1.322265,1.341143,1.346202,1.294336,1.359204,1.328350,1.340989,1.352717,0.0
19371,0.623665,0.564269,0.519484,0.492092,0.561203,0.471161,0.608575,0.581709,0.579173,0.511012,...,1.299334,1.309861,1.358664,1.271001,1.388717,1.361720,1.295100,1.296166,1.350382,0.0
19372,0.548308,0.526311,0.558804,0.603206,0.437835,0.499259,0.615961,0.377607,0.531348,0.441672,...,1.347398,1.200716,1.251561,1.318744,1.269002,1.226858,1.221599,1.275983,1.329643,0.0


In [32]:
feature_df8

,0,1,2,3,4,5,6,7,8,9,...,9,10,11,12,13,14,15,16,17,9216
0,0.335164,0.422834,0.425434,0.386857,0.268384,0.364790,0.372324,0.337670,0.317918,0.323084,...,0.547509,0.428001,0.381886,0.538637,0.495738,0.543177,0.453424,0.320640,1.318853,0.0
1,0.558940,0.547185,0.536815,0.439565,0.635416,0.497883,0.601128,0.535507,0.491099,0.487306,...,1.198504,1.258176,1.284958,1.187919,1.312455,1.242558,1.224121,1.272901,1.324214,0.0
2,0.481070,0.521079,0.499746,0.613020,0.501244,0.668536,0.593137,0.620548,0.625243,0.594081,...,1.187891,1.316769,1.341679,1.294233,1.312466,1.173382,1.296258,1.354735,1.374708,0.0
3,0.648244,0.654992,0.654824,0.651539,0.498437,0.690035,0.509310,0.669250,0.483494,0.501346,...,0.837796,0.424594,0.528106,0.817291,0.404327,0.634072,0.440675,0.732804,1.354924,1.0
4,0.273979,0.277746,0.284761,0.281482,0.303147,0.285764,0.280198,0.274268,0.269927,0.271021,...,0.690006,0.812859,0.717739,0.951507,0.772926,0.886945,0.730272,0.576406,1.311020,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0.420159,0.539634,0.553114,0.445364,0.456413,0.484084,0.491127,0.502279,0.405376,0.530856,...,1.272673,1.261968,1.310505,1.349567,1.252342,1.353315,1.225385,1.336892,1.304636,0.0
19370,0.481627,0.467439,0.593716,0.594382,0.465957,0.529134,0.553749,0.482576,0.462229,0.442401,...,1.268374,1.322265,1.341143,1.346202,1.294336,1.359204,1.328350,1.340989,1.352717,0.0
19371,0.623665,0.564269,0.519484,0.492092,0.561203,0.471161,0.608575,0.581709,0.579173,0.511012,...,1.299334,1.309861,1.358664,1.271001,1.388717,1.361720,1.295100,1.296166,1.350382,0.0
19372,0.548308,0.526311,0.558804,0.603206,0.437835,0.499259,0.615961,0.377607,0.531348,0.441672,...,1.347398,1.200716,1.251561,1.318744,1.269002,1.226858,1.221599,1.275983,1.329643,0.0


In [33]:
feature_df8.to_csv('Full_feature_data16sec_15_feature.csv', index = False)

In [22]:
feature_df8.columns = [i for i in range(199)]
feature_df8.columns

ValueError: Length mismatch: Expected axis has 271 elements, new values have 199 elements

In [26]:
np.isinf(feature_df8).values.any()

True

In [ ]:
feature_df8 = feature_df8.replace([np.inf, -np.inf], np.nan)
feature_df8.dropna(inplace = True)

In [ ]:
feature_df8.reset_index(drop = True, inplace = True)

## Principal Component Analysis (PCA):

In [20]:
# Imports

from sklearn.decomposition import PCA

In [21]:
# Set hyperparams for PCA

n_components = 20
random_state = 32

### PCA 20

In [22]:
pca_20 = PCA(n_components = n_components, random_state = random_state)
feature_df_20 = pca_20.fit_transform(feature_df8[feature_df8.columns[:-1]])
# feature_df_20 = pca_20.fit_transform(feature_df1[feature_df1[[-1]]])

In [23]:
pca_20_df = pd.DataFrame(data = feature_df_20)
pca_20_df = pd.concat([pca_20_df, feature_df8[feature_df8.columns[-1]]], axis = 1)
pca_20_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,198
0,-0.698401,0.711614,-0.535396,-2.306384,-5.390280,-3.300912,-3.536100,0.565532,0.824587,1.508695,...,-1.313258,-0.496623,-1.041469,-0.915638,0.240926,0.668054,-0.675300,-0.301267,0.302881,0.0
1,38.632791,5.813591,-4.760762,-3.675133,-2.098731,1.304465,8.514638,2.862905,6.584114,-0.879614,...,0.669272,-2.839235,5.144056,3.021264,1.135477,4.938117,-3.944832,-2.032339,0.142766,0.0
2,-3.015155,1.486673,-0.489488,-2.556610,-1.350293,1.304915,-0.779178,-0.482346,1.026748,0.864856,...,-0.199581,-0.235266,0.297306,-0.935886,-0.722313,1.408623,0.107452,1.213886,0.873866,0.0
3,-4.634571,0.056001,0.432330,-2.822244,1.340311,0.226598,0.469524,0.528140,-0.986277,-0.579865,...,-0.673840,0.588405,1.241491,0.060544,-0.390271,1.183287,-0.163433,-1.051416,-0.325579,0.0
4,-5.663073,0.678729,0.994924,-3.925916,0.026382,0.562499,-0.009809,-0.293030,-0.159686,0.251238,...,-1.133158,-0.648158,0.167079,-0.631026,-0.192914,0.973578,0.347587,-0.731707,0.210178,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38766,-7.863200,0.174535,0.879323,-0.615529,-0.672648,0.418831,0.595452,0.026663,-0.137261,0.385012,...,-0.504359,-0.418479,0.525209,0.298484,-0.192581,0.371103,0.125411,0.724511,0.178908,0.0
38767,-3.716091,-0.587680,0.672288,-0.061045,0.226207,0.100715,-1.745069,0.478802,-0.023813,0.035862,...,-0.171394,-1.543685,0.780052,-0.291897,-0.236894,0.202725,0.348269,-0.127580,0.425756,0.0
38768,-6.983744,-0.021924,-0.492173,-1.473092,1.115756,-0.847503,0.614432,-0.120307,-0.177788,-0.010997,...,-0.221306,0.604943,0.181909,-0.433279,0.068443,0.007763,-0.070160,-0.835676,0.205445,0.0
38769,16.422236,2.135077,1.681675,2.642308,-0.656844,-0.299316,0.731701,0.421246,0.852216,-0.898412,...,0.077857,0.452812,0.308966,1.035972,-0.457531,0.850859,-2.370629,-2.371177,0.472789,0.0


In [24]:
pca_20_df.to_csv('8sec/PCA_20_features.csv', index = False)

### PCA 50

In [ ]:
n_components = 50

pca_50 = PCA(n_components = n_components, random_state = random_state)
feature_df_50 = pca_50.fit_transform(feature_df8[feature_df8.columns[:-1]])

pca_50_df = pd.DataFrame(data = feature_df_50)
pca_50_df = pd.concat([pca_50_df, feature_df8[feature_df8.columns[-1]]], axis = 1)
pca_50_df

In [ ]:
pca_50_df.to_csv('8sec/PCA_50_features.csv', index = False)

### PCA 70

In [ ]:
n_components = 70

pca_70 = PCA(n_components = n_components, random_state = random_state)
feature_df_70 = pca_70.fit_transform(feature_df8[feature_df8.columns[:-1]])

pca_70_df = pd.DataFrame(data = feature_df_70)
pca_70_df = pd.concat([pca_70_df, feature_df8[feature_df8.columns[-1]]], axis = 1)

pca_70_df.to_csv('8sec/PCA_70_features.csv', index = False)

### PCA 100

In [ ]:
n_components = 100

pca_100 = PCA(n_components = n_components, random_state = random_state)
feature_df_100 = pca_100.fit_transform(feature_df8[feature_df8.columns[:-1]])

pca_100_df = pd.DataFrame(data = feature_df_100) 
pca_100_df = pd.concat([pca_100_df, feature_df8[feature_df8.columns[-1]]], axis = 1)

pca_100_df.to_csv('8sec/PCA_100_features.csv', index = False)